## Few-Shot-Learning


### Divisão dos Dados

Cria um dicionário dos dados (class map): {label_classe: [lista de  imagens dessa classe]} e divide os dados entre treino e teste

**Estrutura do Omniglot**


omniglot-py/ <br>
└── images_background/  (ou images_evaluation/)<br>
    ├── Alphabet_of_the_Magi/<br>
    │   ├── character01/<br>
    │   │   ├── 0709_01.png<br>
    │   │   ├── 0709_02.png<br>
    │   │   └── ... (20 imagens no total)<br>
    │   └── character02/<br>
    │       └── ...<br>
    ├── Angelic/<br>
    │   └── ...<br>
    └── ... (mais 30 alfabetos)<br>

In [1]:
import torchvision as tv
from torchvision import transforms

O Omniglot é dividido em background e evaluation. O conjunto background contém os alfabetos para treinar o seu modelo a "aprender a aprender". O conjunto evaluation contém alfabetos completamente novos, que o modelo nunca viu, e é usado para testar se ele consegue generalizar para novas classes com poucos exemplos.

### Episódios

- Em vez de treinar o modelo para ser um especialista em um conjunto fixo de classes, nós o treinamos para ser um generalista em aprender.
- Seria dar ao modelo uma série de "mini testes surpresa" (os episódios). Cada teste é uma pequena tarefa de classificação.

**Anatomia de um Episódio**

Em um cenário N-way K-shot (onde N = 5 e K = 1):

1. **Amostragem:** Primeiro, escolhemos aleatoriamente N classes do nosso enorme conjunto de dados de treinamento. 
2. **Support Set:** Para cada uma dessas N classes, damos ao modelo K exemplos. Este é o "material de estudo" para este teste específico. O modelo deve olhar para essas "N times K" imagens e aprender a distinguir as N classes.
3. **Query Set:** Em seguida, pegamos outras imagens (que não estavam no material de estudo) daquelas mesmas N classes. Estas são as "perguntas do teste".
4. **Avaliação e Aprendizado:** O modelo tenta classificar as imagens do Query Set. Calculamos o quão bem ele se saiu (loss). Usamos essa nota para ajustar os pesos do modelo através do backpropagation.


Ao repetir esse processo milhares de vezes, com milhares de combinações diferentes de classes, o modelo não está memorizando "o que é o caractere A do alfabeto Grego". Em vez disso, ele está aprendendo uma estratégia para, dado qualquer conjunto de N novas classes com K exemplos cada, descobrir as características que as diferenciam.

In [3]:
import torch, random
import numpy as np


def create_episode(classes, n_way, k_shot, k_query):
    episode_classes_idx = random.sample(list(classes.keys()), n_way)
    
    support_set = []
    support_labels = []
    query_set = []
    query_labels = []
    
    
    #* Montar o Support e o Query set para cada classe selecionada
    for i, class_idx in enumerate(episode_classes_idx):
        images_of_class = classes[class_idx] 
        
        #* Sortear imagens dessa classe
        selected_images = random.sample(images_of_class, k_shot + k_query)
        
        #* Dividir as imagens sorteadas nos dois conjuntos
        support_images = selected_images[:k_shot]
        query_images = selected_images[k_shot:]
        
        support_set.extend(support_images)
        query_set.extend(query_images)
        
        #* Criar as labels RELATIVAS ao episódio
        support_labels.extend([i] * k_shot)
        query_labels.extend([i] * k_query)


    #* Embaralhar os conjuntos
    s_indices = np.random.permutation(len(support_set))
    q_indices = np.random.permutation(len(query_set))
    
    #* Reordenar usando os índices embaralhados
    support_set = torch.stack(support_set)[s_indices]
    support_labels = torch.tensor(support_labels)[s_indices]
    query_set = torch.stack(query_set)[q_indices]
    query_labels = torch.tensor(query_labels)[q_indices]
    
    #*Retorna os 4 tensores para serem usados pelo modelo
    return support_set, support_labels, query_set, query_labels
    


Um tensor é uma estrutura de dados usada para armazenar números em múltiplas dimensões, muito comum em machine learning e deep learning.

### Redes Prototípicas

**A Analogia: O Crítico de Arte Novato**

Imagine que seu modelo é um crítico de arte novato. Cada "episódio" é um novo desafio para ele.

1. O Desafio (Episódio): Levamos o crítico a uma galeria com 5 artistas que ele nunca viu antes (N_WAY = 5).

2. O Material de Estudo (support_set): Mostramos a ele apenas uma pintura de cada um desses 5 artistas (K_SHOT = 1). A tarefa dele é estudar essas 5 pinturas e formar uma ideia central do "estilo" de cada artista.

3. A Ideia Central (Protótipo): Depois de olhar a pintura de um artista, o crítico cria uma "representação mental" do estilo daquele artista. Essa representação é o protótipo. Ele faz isso para os 5 artistas.

4. O Teste (query_set): Agora, mostramos ao crítico uma nova pintura (que ele não usou para estudo) e perguntamos: "Qual dos 5 artistas pintou esta obra?".

5. A Resposta (Classificação): O crítico compara a nova pintura com as 5 "representações mentais" (protótipos) que ele criou. A que for mais parecida, ele supõe que seja do mesmo artista.

6. O Aprendizado (Loss & Optimizer): Se ele errar, nós o corrigimos. Essa correção o ajuda a refinar sua capacidade de extrair a "essência" de uma pintura, para que da próxima vez ele forme representações mentais melhores.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm


#* Transforma uma imagem em um vetor de características
class EmbeddingNet(nn.Module):
    def __init__(self, embedding_dim=64):
        super(EmbeddingNet, self).__init__()
        self.convnet = nn.Sequential(
            #* Primeiro bloco convolucional
            nn.Conv2d(1, 64, kernel_size=3, padding=1),  # Aplica uma convolução 2D com 1 canal de entrada, 64 canais de saída, kernel 3x3 e padding de 1.
            nn.BatchNorm2d(64),                          # Normaliza a saída da convolução para estabilizar o treinamento.
            nn.ReLU(),                                   # Aplica a função de ativação ReLU para introduzir não-linearidade.
            nn.MaxPool2d(2),                             # Reduz as dimensões espaciais aplicando pooling máximo 2x2.

            #* Segundo bloco convolucional
            nn.Conv2d(64, 64, kernel_size=3, padding=1), # Outra convolução 2D com 64 canais de entrada e saída, kernel 3x3 e padding de 1.
            nn.BatchNorm2d(64),                          # Normaliza a saída da segunda convolução.
            nn.ReLU(),                                   # Aplica novamente a função de ativação ReLU.
            nn.MaxPool2d(2),                             # Reduz ainda mais as dimensões espaciais com outro pooling máximo 2x2.

            #* Terceiro bloco convolucional
            nn.Conv2d(64, 64, kernel_size=3, padding=1), # Terceira convolução 2D com a mesma configuração.
            nn.BatchNorm2d(64),                          # Normaliza a saída da terceira convolução.
            nn.ReLU(),                                   # Aplica mais uma vez a função de ativação ReLU.
            nn.MaxPool2d(2)                              # Pooling máximo final 2x2 para reduzir ainda mais as dimensões espaciais.
        )
        self.fc = nn.Sequential(
            nn.Flatten(),  # Transforma a saída 3D (batch_size, canais, altura, largura) em uma saída 2D (batch_size, canais * altura * largura).
            nn.Linear(64 * 3 * 3, embedding_dim)  # Aplica uma camada totalmente conectada (fully connected) que reduz a dimensão de entrada (64 * 3 * 3) para o tamanho do embedding especificado (embedding_dim).
        )
    
    def forward(self, x):
        x = self.convnet(x)
        x = self.fc(x)
        return x

- "Convolução 2D": É uma operação matemática aplicada a imagens (que são matrizes 2D de pixels) para extrair características, como bordas ou texturas.
- "1 canal de entrada": A imagem de entrada tem 1 canal, ou seja, é uma imagem em preto e branco (escala de cinza).
- "64 canais de saída": A camada gera 64 imagens de saída (chamadas de mapas de ativação ou feature maps), cada uma destacando diferentes padrões encontrados na imagem.
- "kernel 3x3": O filtro (ou janela) que percorre a imagem tem tamanho 3x3 pixels. Ele examina pequenos blocos da imagem de cada vez.
- "padding de 1": Adiciona uma borda de 1 pixel ao redor da imagem para garantir que a saída tenha o mesmo tamanho que a entrada (ou quase).

A rede é composta por três blocos de camadas convolucionais (Conv2d), cada um seguido por normalização em lote (BatchNorm2d), função de ativação ReLU e uma camada de pooling (MaxPool2d). O objetivo dessa sequência é extrair representações compactas e discriminativas das imagens, reduzindo gradualmente sua dimensão espacial e aumentando a profundidade das características aprendidas.

### Treino

In [5]:
#* Preparar dados de treino

transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor()
])

omniglot_train = tv.datasets.Omniglot(
    root="../data",
    background=True,
    download=True,
    transform=transform
)

100%|██████████| 9.46M/9.46M [00:01<00:00, 4.83MB/s]


In [6]:
#* Agrupar imagens por classe, facilitando a amostragem

train_class_map = {}
for img, label in omniglot_train:
    if label not in train_class_map:
        train_class_map[label] = []
    train_class_map[label].append(img)
    

In [ ]:
#* Configurações de treinamento
N_WAY = 5
K_SHOT = 1
K_QUERY = 15
NUM_EPS = 1000
EMBEDDING_DIM = 64


#* Inicializando modelo e otimizador
model = EmbeddingNet(embedding_dim=EMBEDDING_DIM)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

- **embedding_dim:** é a quantidade de números (dimensão) que representa cada imagem no "espaço de características" aprendido pela rede.
<br>Quanto maior o embedding_dim, mais informações a rede pode guardar sobre cada imagem, mas também aumenta a complexidade do modelo.

In [ ]:
#* Loop de treinamento
print("Training...")

for episode in tqdm(range(NUM_EPS)):
    #* Pegando um novo episódio 
    support_x, support_y, query_x, query_y  = create_episode(train_class_map, N_WAY, K_SHOT, K_QUERY)
    
    #* Zerar gradientes do otimizador
    optimizer.zero_grad()
    
    #* Passar imagens de suporte e de query (teste) 
    support_embeddings = model(support_x) # Gera as representações "mentais"
    query_embeddings = model(query_x)
    
    #* Calcular o protótipo de cada classe
    prototypes = []
    for i in range(N_WAY):  # Itera sobre cada classe na tarefa N-way
        class_embeddings = support_embeddings[support_y == i]  # Seleciona as embeddings correspondentes à classe atual
        prototypes.append(class_embeddings.mean(dim=0))  # Calcula a média das embeddings da classe e adiciona à lista de protótipos
    prototypes = torch.stack(prototypes)  # Empilha todos os protótipos de classe em um único tensor
    
    #* Calcular a distancia de cada imagem de teste para cada prorótipo
    distances = torch.cdist(query_embeddings, prototypes, p=2).pow(2) # Euclideana ao quadrado
    
    #* Calcular a perda do modelo
    log_p_y = F.log_softmax(-distances, dim=1)
    loss = F.nll_loss(log_p_y, query_y)
    
    #*Realiza a correção
    loss.backward()
    optimizer.step()

print("Train is over")


    

Training...


100%|██████████| 1000/1000 [01:22<00:00, 12.16it/s]

Train is over


In [15]:
torch.save(model.state_dict(), "prototypical_net.pth")

### Teste

In [16]:
#* Carregando o conjunto de teste

transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor()
])


omniglot_test = tv.datasets.Omniglot(
    root="../data",
    background=False, 
    download=True,
    transform=transform
)

test_class_map = {}
for img, label in omniglot_test:
    if label not in test_class_map:
        test_class_map[label] = []
    test_class_map[label].append(img)

In [ ]:
#* Avaliando o modelo

model.load_state_dict(torch.load("prototypical_net.pth", weights_only=False))
model.eval()

# Essa linha carrega os pesos (parâmetros treinados) do modelo salvos anteriormente no arquivo 
# "prototypical_net.pth" para dentro do modelo atual.

EmbeddingNet(
  (convnet): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=576, out_features=64, bias=True)
  )
)

In [ ]:
NUM_EPS_TESTE = 1000
accuracies = []

print("Testing model...")
with torch.no_grad(): #* Não se preocupa com gradientes, e sim apenas em testar
    for episode in tqdm(range(NUM_EPS_TESTE)):
        support_x, support_y, query_x, query_y = create_episode(test_class_map, N_WAY, K_SHOT, K_QUERY)
        
        support_embeddings = model(support_x)
        query_embeddings = model(query_x)
        
        prototypes = []
        for i in range(N_WAY):
            class_embeddings = support_embeddings[support_y == i]
            prototypes.append(class_embeddings.mean(dim=0))
        prototypes = torch.stack(prototypes)
        
        distances = torch.cdist(query_embeddings, prototypes, p=2).pow(2)
        
        #* torch.argmin encontra o índice da menor distância, que é a nossa classe prevista
        predictions = torch.argmin(distances, dim=1)
        
        #* Comparamos as predições com as labels verdadeiras para calcular a acurácia do episódio
        #* (predictions == query_y) -> Tensor de True/False
        #* .float() -> Converte para 1.0/0.0
        #* .mean() -> Calcula a média (ex: 60 acertos em 75 -> 0.80)
        #* .item() -> Extrai o valor numérico do tensor
        accuracy = (predictions == query_y).float().mean().item()
        accuracies.append(accuracy)

print("Test is over")

100%|██████████| 1000/1000 [00:36<00:00, 27.26it/s]


## Resultados

In [20]:
mean_accuracy = np.mean(accuracies)
std_accuracy = np.std(accuracies)
confidence_interval = 1.96 * std_accuracy / np.sqrt(NUM_EPS_TESTE) # Fórmula do IC 95%

print("\n--- Resultados da Avaliação ---")
print(f"Acurácia Média: {mean_accuracy * 100:.2f}%")
print(f"Intervalo de Confiança (95%): ± {confidence_interval * 100:.2f}%")
print(f"Resultado Final: {mean_accuracy * 100:.2f}% ± {confidence_interval * 100:.2f}%")


--- Resultados da Avaliação ---
Acurácia Média: 90.96%
Intervalo de Confiança (95%): ± 0.45%
Resultado Final: 90.96% ± 0.45%
